<a href="https://colab.research.google.com/github/siddharth-chakravarthula/IPL-Win-Predictor/blob/main/iplwinpredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sidtheimmortalgod","key":"e3dd366b6aa3e0f28313c6593a04ada1"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nowke9/ipldata

Dataset URL: https://www.kaggle.com/datasets/nowke9/ipldata
License(s): CC-BY-NC-SA-4.0
ipldata.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip ipldata.zip

Archive:  ipldata.zip
replace deliveries.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: deliveries.csv          
replace matches.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: matches.csv             


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

ipl_match_data = pd.read_csv('matches.csv')
ipl_match_data.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
ipl_match_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               756 non-null    int64 
 1   season           756 non-null    int64 
 2   city             749 non-null    object
 3   date             756 non-null    object
 4   team1            756 non-null    object
 5   team2            756 non-null    object
 6   toss_winner      756 non-null    object
 7   toss_decision    756 non-null    object
 8   result           756 non-null    object
 9   dl_applied       756 non-null    int64 
 10  winner           752 non-null    object
 11  win_by_runs      756 non-null    int64 
 12  win_by_wickets   756 non-null    int64 
 13  player_of_match  752 non-null    object
 14  venue            756 non-null    object
 15  umpire1          754 non-null    object
 16  umpire2          754 non-null    object
 17  umpire3          119 non-null    ob

In [ ]:
ipl_match_data.team1.unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [ ]:
# Delhi Daredevils and Capitals are the same team with the same managemnet, just with different branding, so we will set Daredevils equaal to Capitals.
ipl_match_data['team1']=ipl_data['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
ipl_match_data['team2']=ipl_data['team2'].str.replace('Delhi Daredevils','Delhi Capitals')
ipl_match_data['winner']=ipl_data['winner'].str.replace('Delhi Daredevils','Delhi Capitals')
ipl_match_data.team1.unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Capitals', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants'],
      dtype=object)

In [ ]:
ipl_ball_data = pd.read_csv('deliveries.csv')
ipl_ball_data.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [ ]:
ipl_ball_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          179078 non-null  int64 
 1   inning            179078 non-null  int64 
 2   batting_team      179078 non-null  object
 3   bowling_team      179078 non-null  object
 4   over              179078 non-null  int64 
 5   ball              179078 non-null  int64 
 6   batsman           179078 non-null  object
 7   non_striker       179078 non-null  object
 8   bowler            179078 non-null  object
 9   is_super_over     179078 non-null  int64 
 10  wide_runs         179078 non-null  int64 
 11  bye_runs          179078 non-null  int64 
 12  legbye_runs       179078 non-null  int64 
 13  noball_runs       179078 non-null  int64 
 14  penalty_runs      179078 non-null  int64 
 15  batsman_runs      179078 non-null  int64 
 16  extra_runs        179078 non-null  int

In [ ]:
# Data Preprocessing
ipl_match_data.rename(columns={'id': 'match_id'}, inplace=True)

# Merge Ball and Match Data
ipl_merged_data = pd.merge(ipl_match_data, ipl_ball_data, on='match_id', how='inner')

# Filter for 2nd Innings Data
ipl_merged_data = ipl_merged_data[ipl_merged_data['inning'] == 2]
ipl_merged_data['runs_remaining'] = ipl_merged_data.groupby('match_id')['total_runs'].transform('sum') - ipl_merged_data['total_runs'].cumsum()
ipl_merged_data['balls_remaining'] = 120 - (ipl_merged_data['over'] * 6 + ipl_merged_data['ball'])
ipl_merged_data.head()

,match_id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,runs_remaining,balls_remaining
125,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,...,0,0,1,0,1,NaN,NaN,NaN,171,113
126,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,...,0,0,0,0,0,NaN,NaN,NaN,171,112
127,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,...,0,0,0,0,0,NaN,NaN,NaN,171,111
128,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,...,0,0,2,0,2,NaN,NaN,NaN,169,110
129,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,...,0,0,4,0,4,NaN,NaN,NaN,165,109


In [ ]:
# Feature Engineering
ipl_merged_data['wickets_lost'] = ipl_merged_data.groupby('match_id')['player_dismissed'].cumcount()
ipl_merged_data['win'] = ipl_merged_data['winner'] == ipl_merged_data['batting_team']
ipl_merged_data['win'] = ipl_merged_data['win'].astype(int)

In [ ]:
# Create Train and Test Sets
features = ['batting_team', 'bowling_team', 'venue', 'runs_remaining', 'balls_remaining', 'wickets_lost']
X = ipl_merged_data[features]
y = ipl_merged_data['win']

# Encode Categorical Features
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Encode Categorical Data
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X[['batting_team', 'bowling_team', 'venue']])

# Normalize Numerical Data
scaler = StandardScaler()
X_numerical = scaler.fit_transform(X[['runs_remaining', 'balls_remaining', 'wickets_lost']])

# Combine the Categorical and Numeric Columns
X = np.hstack([X_encoded, X_numerical])

# Creation of Train and Test Sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the Encoder and Scaler
import joblib
joblib.dump(encoder, 'encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
# Build the Model

import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

# Save the Model
model.save("ipl_win_probability_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7587 - loss: 0.4562 - val_accuracy: 0.9310 - val_loss: 0.1522
Epoch 2/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9209 - loss: 0.1733 - val_accuracy: 0.9507 - val_loss: 0.1068
Epoch 3/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9441 - loss: 0.1240 - val_accuracy: 0.9701 - val_loss: 0.0743
Epoch 4/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9586 - loss: 0.0933 - val_accuracy: 0.9781 - val_loss: 0.0545
Epoch 5/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9707 - loss: 0.0708 - val_accuracy: 0.9892 - val_loss: 0.0382
Epoch 6/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9776 - loss: 0.0547 - val_accuracy: 0.9894 - val_loss: 0.0324
Epoch 7/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9834 - loss: 0.0424 - val_accuracy: 0.9943 - val_loss: 0.0241
Epoch 8/10
2156/2156 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9850 - loss: 0.0366 -

In [ ]:
def interpret_user_input(encoder, scaler):
    """
    Collects, validates, and processes user inputs for IPL win probability prediction.

    Args:
        encoder (OneHotEncoder): Fitted OneHotEncoder for categorical data.
        scaler (StandardScaler): Fitted StandardScaler for numerical data.

    Returns:
        np.ndarray: Preprocessed input data ready for prediction.
    """

    valid_teams = [
        'Mumbai Indians', 'Chennai Super Kings', 'Delhi Capitals',
        'Kolkata Knight Riders', 'Punjab Kings', 'Rajasthan Royals',
        'Royal Challengers Bangalore', 'Sunrisers Hyderabad', 'Lucknow Super Giants',
        'Gujarat Titans'
    ]
    valid_venues = [
        'Wankhede Stadium', 'Eden Gardens', 'M. Chinnaswamy Stadium',
        'Narendra Modi Stadium', 'Feroz Shah Kotla Ground',
        'Sawai Mansingh Stadium', 'Arun Jaitley Stadium'
    ]

    print("Provide the details for IPL chase scenario prediction:")

    # Prompt for Categorical Data
    batting_team = input("Enter batting team: ")
    while batting_team not in valid_teams:
        print(f"Invalid team. Please choose from: {valid_teams}")
        batting_team = input("Enter batting team: ")

    bowling_team = input("Enter bowling team: ")
    while bowling_team not in valid_teams:
        print(f"Invalid team. Please choose from: {valid_teams}")
        bowling_team = input("Enter bowling team: ")

    venue = input("Enter venue: ")
    while venue not in valid_venues:
        print(f"Invalid venue. Please choose from: {valid_venues}")
        venue = input("Enter venue: ")

    # Prompt for Numerical Details
    try:
        runs_remaining = int(input("Enter runs remaining: "))
        balls_remaining = int(input("Enter balls remaining: "))
        wickets_lost = int(input("Enter wickets lost (0-10): "))
        if wickets_lost < 0 or wickets_lost > 10 or balls_remaining > 120:
            raise ValueError
    except ValueError:
        print("Invalid numerical input. Please enter integers within the range of the T20 format only.")
        return interpret_user_input(encoder, scaler)

    # Preprocess Categorial and Numerical Inputs
    categorical_features = np.array([[batting_team, bowling_team, venue]])
    encoded_categorical = encoder.transform(categorical_features)

    numerical_features = np.array([[runs_remaining, balls_remaining, wickets_lost]])
    normalized_numerical = scaler.transform(numerical_features)

    # Combine Final Inputs
    final_input = np.hstack([encoded_categorical, normalized_numerical])
    return final_input

In [ ]:
# Using the Function through the Console
user_data = interpret_user_input(encoder, scaler)
prediction = model.predict(user_data)
print(f"Win Percentage for the Batting Team: {prediction[0][0]}")


Provide the details for IPL chase scenario prediction:


KeyboardInterrupt: Interrupted by user